## Metrics with new threshold

In [205]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix,  auc, precision_recall_curve, brier_score_loss

In [ ]:
df = pd.read_csv("./data/SNUH/auc_f1.csv")

In [ ]:
df

,new_total_aki,transformer_val,xgb_val,RF_val,LR_val,svc_val,transformer+notext_val
0,0,0.047404,0.074181,0.084133,0.110741,0.115589,0.015922
1,0,0.161916,0.134149,0.152761,0.105839,0.005829,0.134435
2,0,0.019681,0.020899,0.004926,0.019550,0.023344,0.006488
3,0,0.011778,0.015613,0.004830,0.037785,0.023518,0.009038
4,0,0.026032,0.006413,0.058442,0.021802,0.095524,0.004110
...,...,...,...,...,...,...,...
13755,0,0.036322,0.034521,0.030859,0.052571,0.028645,0.033590
13756,0,0.162779,0.056739,0.100516,0.036026,0.016496,0.197097
13757,0,0.169887,0.080851,0.226609,0.103048,0.248647,0.088899
13758,0,0.414069,0.301351,0.252989,0.259893,0.032560,0.252568


In [172]:
#f1 : 0.094 /auroc :0.039
#transformer_val: 0.41
#xgb_val : 0.18
#RF_val : 0.21
#LR_val : 0.11
#svc_val : 0.14
#transformer+notext_val: 0.19

### find optimal threshold

In [280]:
col = "transformer_val"
y_test = df['new_total_aki']

In [281]:
cutoff  = list(np.arange(0.0,1.0,0.01))

In [ ]:
l = {}
for c in cutoff :    
    roc_predictions = [1 if i >= c else 0 for i in df[col]]
    l.update({round(c,2): round(f1_score(y_test, roc_predictions),4)})

In [283]:
optimal_threshold = max(l, key=l.get)

0.41

In [179]:
# SNUH : check metrics
optimal_proba_cutoff = 0.19
roc_predictions = [1 if i >= optimal_proba_cutoff else 0 for i in df[col]]
predictions = [1 if i >= 0.5 else 0 for i in df[col]]
y_test = df['new_total_aki']

print("Accuracy Score Before and After Thresholding: {}, {}".format(round(accuracy_score(y_test, predictions),4), round(accuracy_score(y_test, roc_predictions),4)))
print("Precision Score Before and After Thresholding: {}, {}".format(round(precision_score(y_test, predictions),4), round(precision_score(y_test, roc_predictions),4)))
print("Recall Score Before and After Thresholding: {}, {}".format(round(recall_score(y_test, predictions),4), round(recall_score(y_test, roc_predictions),4)))
print("F1 Score Before and After Thresholding: {}, {}".format(round(f1_score(y_test, predictions),4), round(f1_score(y_test, roc_predictions),4)))

Accuracy Score Before and After Thresholding: 0.9443, 0.9081
Precision Score Before and After Thresholding: 0.497, 0.3031
Recall Score Before and After Thresholding: 0.1085, 0.502
F1 Score Before and After Thresholding: 0.1781, 0.378


In [180]:
# KMC : check metrics

dk = pd.read_csv("./data/KMC/auc_f1.csv")
y_test = dk['new_total_aki']

#optimal_proba_cutoff = 0.41
roc_predictions = [1 if i >= optimal_proba_cutoff else 0 for i in dk['transformer_val']]
predictions = [1 if i >= 0.5 else 0 for i in dk['transformer_val']]

precision, recall, threshold = precision_recall_curve(y_test, dk['transformer_val'])

# precision = precision_score(y_test, predictions)
# recall = recall_score(y_test, predictions)

auprc = auc(recall, precision)

print("Accuracy Score Before and After Thresholding: {}, {}".format(round(accuracy_score(y_test, predictions),4), round(accuracy_score(y_test, roc_predictions),4)))
print("Precision Score Before and After Thresholding: {}, {}".format(round(precision_score(y_test, predictions),4), round(precision_score(y_test, roc_predictions),4)))
print("Recall Score Before and After Thresholding: {}, {}".format(round(recall_score(y_test, predictions),4), round(recall_score(y_test, roc_predictions),4)))
print("F1 Score Before and After Thresholding: {}, {}".format(round(f1_score(y_test, predictions),4), round(f1_score(y_test, roc_predictions),4)))
print("AUPRC Before and After Thresholding: {}, {}".format(round(auc(recall, precision),4), round(auc(recall, precision),4)))

Accuracy Score Before and After Thresholding: 0.9337, 0.9181
Precision Score Before and After Thresholding: 0.4703, 0.29
Recall Score Before and After Thresholding: 0.0424, 0.1662
F1 Score Before and After Thresholding: 0.0778, 0.2113
AUPRC Before and After Thresholding: 0.1825, 0.1825


### Metrics with new threshold

In [434]:
metrics = ["auroc", "f1", "specificity", "sensitivity"]


round_digits  = 3
import stat_util

def new_metric(task):
    
    for m in range(len(metrics)) :
        print(metrics[m])
        #import data & threshold
        
        if task == "SNUH" :
            df = pd.read_csv("./data/SNUH/"+metrics[m]+"/auc.csv") #SNUH -> calculate optimal threshold 
            res = {'transformer_val': 0.41,
                     'xgb_val': 0.18,
                     'RF_val': 0.21,
                     'LR_val': 0.11,
                     'svc_val': 0.14,
                     'transformer+notext_val': 0.19}
        elif task == "SNUH_test" :
            df = pd.read_csv("./data/SNUH_test/"+metrics[m]+"/auc.csv")  #SNUH_test
            res = {'transformer_test': 0.41,
                    'xgb_test': 0.18,
                    'RF_test': 0.21,
                    'LR_test': 0.11,
                    'svc_test': 0.14,
                     'transformer+notext_test': 0.19}
        elif task == "KMC" :
            df = pd.read_csv("./data/KMC/"+metrics[m]+"/auc.csv")  #KMC
            col_list = df.columns[1:]
            res = {}
            res = {'transformer_val': 0.41,
                     'xgb_val': 0.18,
                     'RF_val': 0.21,
                     'LR_val': 0.11,
                     'svc_val': 0.14,
                     'transformer+notext_val': 0.19}

        new_metrics = []



        for col, cutoff in res.items():

            y_true = df['new_total_aki']
            y_pred =  [1 if i >= cutoff else 0 for i in df[col]]
            y_prob = df[col]

            auroc = round(roc_auc_score(y_true, y_prob), round_digits)
            brier_score = round(brier_score_loss(y_true, y_prob), round_digits)


            precisions, recalls, threshold = precision_recall_curve(y_true, y_prob)


            score_auroc, ci_lower_auroc, ci_upper_auroc, scores_auroc = stat_util.score_ci(y_true, y_prob,
                                                                           score_fun=roc_auc_score,
                                                                           n_bootstraps=1000,
                                                                           seed=42)
            score_prauc, ci_lower_prauc, ci_upper_prauc, scores_prauc = stat_util.score_ci(recalls, precisions, score_fun=auc)

            auc_ci = str(round(score_auroc,3))+ " (" + str(round(ci_lower_auroc,3)) + "-" + str(round(ci_upper_auroc,3)) + ")"
            auprc_ci = str(round(score_prauc, 3)) + " (" + str(round(ci_lower_prauc, 3)) + "-" + str(round(ci_upper_prauc, 3)) + ")"


            tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
            precision = precision_score(y_true, y_pred)
            f1 = f1_score(y_true, y_pred)
            recall = recall_score(y_true, y_pred)
            sensitivity =  recall
            specificity = tn / (tn+fp)
            #auprc = auc(recall, precision)

            
            if  col.split('_')[0] == 'RF' :
                new_name = 'Random Forest'
            elif col.split('_')[0] == 'LR' :
                new_name = 'Logistic Regression'
            elif col.split('_')[0] == 'xgb':
                new_name = 'XGBoost'
            elif col.split('_')[0] == 'svc':
                new_name = 'SVM'
            elif col.split('_')[0] == 'transformer'  :
                new_name = 'Transformer'
            elif col.split('_')[0] == 'transformer+notext' :
                new_name = 'Transformer+notext'

            if col.split('_')[-1] in ( 'train','test') :
                new_name= new_name+" "+ col.split('_')[1]
            
            
            new_metrics.append({ 'name' : new_name,
                 'thresholds' : cutoff,               
                'auroc': round(auroc,3),
                'auroc+ci' : auc_ci,
                'auprc+ci' : auprc_ci,
                'precision': round(precision,3),
                'sensitivity': round(sensitivity,3),
                'specificity' : round(specificity,3),
                'f1': round(f1,3),
                'brier': round(brier_score,3)
            })
        final_metrics =  pd.DataFrame.from_dict(new_metrics)
        final_metrics.to_csv("./output/"+task+"/"+metrics[m]+"/result.csv")
        

### print result to csv

In [418]:
# SNUH
new_metric("SNUH")

auroc
f1
specificity


/Users/jeanchoi/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


sensitivity


In [435]:
# SNUH test
new_metric("SNUH_test")

auroc
f1
specificity


/Users/jeanchoi/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


sensitivity


In [420]:
#KMC
new_metric("KMC")

auroc
f1
specificity


/Users/jeanchoi/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


sensitivity
